In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,roc_curve,auc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OrdinalEncoder

df_train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
df_train.head(10)

In [ ]:
df_train.describe()

In [ ]:
x = df_train.drop(['id','claim'],axis=1)
y = df_train['claim']

df_test.drop(['id'],axis=1,inplace=True)

In [ ]:
from scipy import stats

x['missing_count'] = x[x.columns].isnull().sum(axis=1)
df_test['missing_count'] = df_test[df_test.columns].isna().sum(axis=1)

x['std'] = x[x.columns].std(axis=1)
df_test['std'] = df_test[df_test.columns].std(axis=1)

x['min'] = x[x.columns].min(axis=1)
df_test['min'] = df_test[df_test.columns].min(axis=1)

#x['mean'] = x[x.columns].mean(axis=1)
#df_test['mean'] = df_test[df_test.columns].mean(axis=1)

#x['median'] = x[x.columns].median(axis=1)
#df_test['median'] = df_test[df_test.columns].median(axis=1)

x['sem'] = x[x.columns].sem(axis=1)
df_test['sem'] = df_test[df_test.columns].sem(axis=1)

#x['skew'] = x[x.columns].skew(axis=1)
#df_test['skew'] = df_test[df_test.columns].skew(axis=1)

In [ ]:
x

In [ ]:
skew_fea = x.skew()
skew_fea = [*skew_fea[abs(skew_fea.values) > 1].index]

for feat in skew_fea:
    median = x[feat].median()
    x[feat] = x[feat].fillna(median)
    

    

In [ ]:
skew_fea = df_test.skew()
skew_fea = [*skew_fea[abs(skew_fea.values) > 1].index]

for feat in skew_fea:
    median = df_test[feat].median()
    df_test[feat] = df_test[feat].fillna(median)
    



In [ ]:
im = SimpleImputer(strategy='mean')
im_x = pd.DataFrame(im.fit_transform(x))
im_test = pd.DataFrame(im.transform(df_test))

im_x.columns = x.columns
im_test.columns = df_test.columns

In [ ]:
from sklearn.preprocessing import RobustScaler

ss = StandardScaler()
im_x = ss.fit_transform(im_x)
im_test = ss.transform(im_test)

In [ ]:
best_params = {
    'iterations': 18000, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 5, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'devices' : '0',
    #'verbose' : 0
}

In [ ]:
lgbm_params = {'objective': 'binary',
               'boosting_type': 'gbdt',
               'num_leaves': 6,
               'max_depth': 4,
               'learning_rate': 0.05,
               'n_estimators': 50000,
               'reg_alpha': 25.0,
               'reg_lambda': 76.7,
               'random_state': 0,
               'bagging_seed': 0, 
               'feature_fraction_seed': 0,
               'n_jobs': -1,
               'subsample': 0.98,
               'subsample_freq': 1,
               'colsample_bytree': 0.69,
               'min_child_samples': 54,
               'min_child_weight': 256,
               'metric': 'AUC',
               'verbosity': -1,
               'device_type': 'gpu'
              }

In [ ]:
xgb_params = {
             'max_depth': 3, 
             'learning_rate': 0.021537077920105466, 
             'n_estimators': 15000, 
             'min_child_weight': 150, 
             'gamma': 0.11611920725914951, 
             'alpha': 0.0021839958087869794, 
             'lambda': 0.0018567979557499344,
             'colsample_bytree': 0.7139742731494992,
             'subsample': 0.6258627743440968,
             'tree_method': 'gpu_hist',
             'booster': 'gbtree',
             #'random_state': 228,
             'use_label_encoder': False,
             'eval_metric': 'auc'
             }

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

n_split = 5
pred_test_xgb = 0
pred_test_cbr = 0
pred_test_lgbm = 0
#pred_test_tmp = []
scores_c = []
scores_x = []
scores_l = []
xgb_tmp = []
cbr_tmp = []
lgbm_tmp = []
 
for fold,(i,j) in enumerate(KFold(n_splits=n_split,shuffle=True,random_state=1).split(im_x,y)):
    
    x_train , y_train = im_x[i] , y[i]
    x_valid , y_valid = im_x[j] , y[j]
    
    #cbr = CatBoostClassifier(**best_params)
    #cbr.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],verbose=False)
    
    #pred_valid = cbr.predict_proba(x_valid)
    
    #cbr_tmp.extend(pred_valid[:,1])
    
    #scores_cbr = roc_auc_score(y_valid, pred_valid[:,1])
    #scores_c.append(scores_cbr)
    
    #print(f"Fold_cbr_{fold}:{scores_c}")
    
    ## test
    #pred_test_cbr = pred_test_cbr + cbr.predict_proba(im_test)[:,1]
    
    
    #####################################################
    
    xgb = XGBClassifier(**xgb_params)
    xgb.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],verbose=False)
    
    pred_valid = xgb.predict_proba(x_valid) 
    
    xgb_tmp.extend(pred_valid[:,1])
    
    scores_xgb = roc_auc_score(y_valid, pred_valid[:,1])
    scores_x.append(scores_xgb)
    
    print(f"Fold_xgb_{fold}:{scores_x}")
    
    ## test
    pred_test_xgb = pred_test_xgb + xgb.predict_proba(im_test)[:,1]
    
    
    #####################################################
    
    #lgbm = LGBMClassifier(**lgbm_params)
    
    #lgbm.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],verbose=False)
    
    #pred_valid = lgbm.predict_proba(x_valid) 
    
    #lgbm_tmp.extend(pred_valid[:,1])
    
    #scores_lgbm = roc_auc_score(y_valid, pred_valid[:,1])
    #scores_l.append(scores_lgbm)
    
    #print(f"Fold_lgbm_{fold}:{scores_l}")
    
    ## test
    #pred_test_lgbm = pred_test_lgbm + lgbm.predict_proba(im_test)[:,1]
    
    
    
    
#pred_test_lgbm = pred_test_lgbm / n_split
pred_test_cbr = pred_test_cbr / n_split
pred_test_xgb = pred_test_xgb / n_split

    
print(f'Avg cbr_valid score:{np.mean(scores_c)}')
print(f'Avg xgb_valid score:{np.mean(scores_x)}')
#print(f'Avg lgbm_valid score:{np.mean(scores_l)}')

In [ ]:
pred_test_xgb = pred_test_xgb * 0.6

In [ ]:
pred_test_cbr = pred_test_cbr * 0.4

In [ ]:
pred_test_xgb_cbr = (pred_test_xgb + pred_test_cbr) 

In [ ]:
pred_test_xgb_cbr

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

n_split = 5


pred_test_lc = 0
pred_vote = 0

#cbr = CatBoostClassifier(**best_params)
#lgb = LGBMClassifier(**lgbm_params)
#xgb = XGBClassifier(**xgb_params)

xgb_0 = XGBClassifier(**xgb_params, random_state=17)
xgb_1 = XGBClassifier(**xgb_params, random_state=19)
xgb_2 = XGBClassifier(**xgb_params, random_state=75)
xgb_3 = XGBClassifier(**xgb_params, random_state=124)
xgb_4 = XGBClassifier(**xgb_params, random_state=39)
xgb_5 = XGBClassifier(**xgb_params, random_state=55)
cbc_6 = CatBoostClassifier(**best_params, random_state=67)
cbc_7 = CatBoostClassifier(**best_params, random_state=41)
cbc_8 = CatBoostClassifier(**best_params, random_state=85)
cbc_9 = CatBoostClassifier(**best_params, random_state=97)

estimators = [('xgb_0',xgb_0),('xgb_1',xgb_1),('xgb_2',xgb_2),('xgb_3',xgb_3),('xgb_4',xgb_4),('xgb_5',xgb_5),
              ('cbc_6',cbc_6),('cbc_7',cbc_7),('cbc_8',cbc_8),('cbc_9',cbc_9)]
  
    
for fold,(i,j) in enumerate(KFold(n_splits=n_split,shuffle=True,random_state=1).split(im_x,y)):
    
    x_train , y_train = im_x[i] , y[i]
    x_valid , y_valid = im_x[j] , y[j]
    
    model = VotingClassifier(estimators=estimators,voting='soft',verbose=True)
    model.fit(x_train,y_train)
    
    pred = model.predict_proba(x_valid)
    scores_lc = roc_auc_score(y_valid, pred[:,1])
    print(f"Fold_lc_{fold}:{scores_lc}")
    
    
    pred_vote = pred_vote + model.predict_proba(im_test)[:,1]

    
    
pred_vote = pred_vote / n_split
    
print(f'Avg xgb_voting score:{np.mean(scores_lc)}')      
    

In [ ]:
pred_vote

In [ ]:
solution = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
result = pd.DataFrame()
result['id'] = solution['id']
result['claim'] = pred_vote
result

In [ ]:
#result.to_csv('submit_25.csv',index=False)